In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_xtrain = pd.read_csv('sentiment_5_class_train.csv')
df_xtest = pd.read_csv('sentiment_5_class_test.csv')

In [3]:
df_xtest['Sentiment'].replace([0,1,2,3,4],['1-star','2-star','3-star','4-star','5-star'], inplace = True)
df_xtrain['Sentiment'].replace([0,1,2,3,4],['1-star','2-star','3-star','4-star','5-star'], inplace = True)
#df_xtrain

In [4]:
df_xtrain['Sentiment'].value_counts()
df = pd.concat([df_xtrain, df_xtest])
df.isnull().sum()
#no null values

Phrase       0
Sentiment    0
dtype: int64

In [5]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(df_xtrain)

In [6]:
x_train = df_xtrain.drop(columns='Sentiment')

In [7]:
y_train = df_xtrain['Sentiment']

In [8]:
x_test = df_xtest.drop(columns='Sentiment')
y_test = df_xtest['Sentiment']
#when one column is choosen from dataframe it will be converted to series


In [9]:
y_test.shape

(3678,)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
encoded = TfidfVectorizer()
encoded.fit(x_train['Phrase']) #vocab, calculation
final_Xtrain= encoded.transform(x_train['Phrase'])
final_Xtrain.toarray()
#encoded_fearures.shape


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [11]:
final_Xtest = encoded.transform(x_test['Phrase'])
final_Xtest.toarray()
final_Xtest.shape

(3678, 7115)

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [13]:
many_class_hyper = [

    (SVC,
     {
        'kernel':('linear', 'rbf'), 'C': (1,10,5)
     }
    ),


    (DecisionTreeClassifier,
     {
        'max_depth': (2,3,5,8,6,4),
         'min_samples_leaf' : (12, 8, 2,14) #5% of 242
     }
    ), 



    (RandomForestClassifier ,
     {
        'n_estimators': (10,50,100), 
        'max_features': (1,0.5,0.8, 0.1),
        'min_samples_leaf': (12,1,5,8)
    }
    ),
    
      (ExtraTreesClassifier ,
     {
        'n_estimators': (10,50,100), #should use high number, it is the number of decision tree
        'max_features': (1,0.5,0.8, 0.1),
        'min_samples_leaf': (12,1,5,8)
    }
    ),

    (AdaBoostClassifier,
     {
        'n_estimators':(5,10,20,30),
         'learning_rate': (0.5,0.1,0.001,0.0001)
     }
    ),


    (GradientBoostingClassifier,
     {
        'n_estimators': (10,20,40,45),
        'learning_rate': (0.05,0.1, 0.01, 0.2, 0.5),
        'min_samples_split': (2,3,5)
    }
    )

    ]

In [14]:
from sklearn.metrics import f1_score
def grid_search(classifier,param_grid,final_Xtrain, y_train, final_Xtest, y_test ):    
    clf = GridSearchCV(classifier(random_state = 1), param_grid)
    clf.fit(final_Xtrain, y_train)
    print(clf.best_score_ , clf.best_params_)
    best = clf.best_estimator_
    #clf best estimator again trains the model in best hyperparameter
    test_pred = best.predict(final_Xtest)
    print(f1_score(y_test, test_pred, average='macro'))

In [15]:
for classifier, grid_param in many_class_hyper:
    f1_score_value = grid_search(classifier, grid_param, final_Xtrain,y_train,final_Xtest,y_test)
    print(classifier, 'f1_score', f1_score_value)
    
    
    

In [17]:
model = DecisionTreeClassifier()
model.fit(final_Xtrain, y_train)
pred_value = model.predict(final_Xtest)
print(metrics.classification_report(y_test,pred_value))

              precision    recall  f1-score   support

      1-star       0.57      0.51      0.54       247
      2-star       0.52      0.36      0.43       291
      3-star       0.46      0.68      0.55       469
      4-star       0.68      0.66      0.67      1759
      5-star       0.60      0.55      0.57       912

    accuracy                           0.60      3678
   macro avg       0.56      0.55      0.55      3678
weighted avg       0.61      0.60      0.60      3678

